<a href="https://colab.research.google.com/github/seirah-yang/BootCamp/blob/main/Skin_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Skin Disease Classification [Image Dataset]

1. ResNext 학습하기

2. epoch 마다 accuracy score를 나타내는 그래프를 figsave하세요.

3. graph img를 작업 디렉토리에 저장 후, S3에 업로드

In [1]:
!pip install boto3

In [72]:
!unzip archive.zip

Archive:  archive.zip
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024468.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024470.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024707.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024763.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024771.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024800.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024913.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0024948.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025780.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025953.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025957.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0025992.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0026149.jpg  
  inflating: Split_smol/train/Actinic keratosis/ISIC_0026171.jpg  
  inflating: Split_smol/train/Actinic ke

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []
        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label

data_dir = "/content/Split_smol"
batch_size = 32
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = CustomDataset(os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = CustomDataset(os.path.join(data_dir, 'val'), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


model = models.resnext50_32x4d(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, len(train_dataset.classes))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

torch.save(model.state_dict(), 'resnet_model.pth')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy}%')


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 1.980626474727284
Epoch 2/10, Loss: 1.0991057753562927
Epoch 3/10, Loss: 0.576265265995806
Epoch 4/10, Loss: 0.3660311651500789
Epoch 5/10, Loss: 0.22526859085668216
Epoch 6/10, Loss: 0.12086864154447209
Epoch 7/10, Loss: 0.07059284824539315
Epoch 8/10, Loss: 0.058293725956570015
Epoch 9/10, Loss: 0.05596726853400469
Epoch 10/10, Loss: 0.033480876379392364
Accuracy on test set: 86.74033149171271%


In [1]:
import torch
from torchvision import datasets, transforms
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = []
        for label in range(len(self.classes)):
            class_folder = os.path.join(root_dir, self.classes[label])
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label



In [2]:
data_dir = "/content/Split_smol"
batch_size = 32
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
train_dataset = CustomDataset(os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = CustomDataset(os.path.join(data_dir, 'val'), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
model = models.resnext50_32x4d(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, len(train_dataset.classes))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

torch.save(model.state_dict(), 'resnet_model.pth')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy}%')

Epoch 1/10, Loss: 1.9909719228744507
Epoch 2/10, Loss: 1.1058943948962472
Epoch 3/10, Loss: 0.5676637183536183
Epoch 4/10, Loss: 0.32912367514588614
Epoch 5/10, Loss: 0.20302135260267692
Epoch 6/10, Loss: 0.1395291689444672
Epoch 7/10, Loss: 0.11050686849789186
Epoch 8/10, Loss: 0.06003185099160129
Epoch 9/10, Loss: 0.04306109986183318
Epoch 10/10, Loss: 0.0361459439789707
Accuracy on test set: 83.97790055248619%
